In [22]:
# Normal Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
# Machine Learning Packages
import sklearn
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MultiLabelBinarizer
import torch
import torch.nn as nn

# 0. Dataset Overview
> Reference: https://www.kaggle.com/datasets/parisrohan/credit-score-classification/data

**Dataset**: Credit Score Dataset

**Task**: Given a person’s credit-related information, build a machine learning model that can classify the credit score.

**Columns**:
- `id`:	Unique identifier for each record.
- `customer_id`:	Unique identifier for each customer.
- `month`:	Month of the transaction or record.
- `name`:	Customer’s name.
- `age`:	The customer’s age.
- `ssn`:	Customer’s social security number.
- `occupation`:	The customer’s occupation.
- `annual_income`:	The customer’s annual income.
- `monthly_inhand_salary`:	The customer’s monthly take-home salary.
- `num_bank_accounts`:	Total number of bank accounts owned by the customer.
- `num_credit_card`:	Total number of credit cards held by the customer.
- `interest_rate`:	The interest rate applied to loans or credits.
- `num_of_loan`:	Number of loans the customer has taken.
- `type_of_loan`:	Categories of loans obtained by the customer.
- `delay_from_due_date`:	The delay in payment relative to the due date.
- `num_of_delayed_payment`:	Total instances of late payments made by the customer.
- `changed_credit_limit`:	Adjustments made to the customer’s credit limit.
- `num_credit_inquiries`:	Number of inquiries made regarding the customer's credit.
- `credit_mix`:	The variety of credit types the customer uses (e.g., loans, credit cards).
- `outstanding_debt`:	Total amount of debt the customer currently owes.
- `credit_utilization_ratio`:	Proportion of credit used compared to the total credit limit.
- `credit_history_age`:	Duration of the customer’s credit history.
- `payment_of_min_amount`:	Indicates if the customer pays the minimum required amount each month.
- `total_emi_per_month`:	Total Equated Monthly Installment (EMI) paid by the customer.
- `amount_invested_monthly`:	Monthly investment amount made by the customer.
- `payment_behaviour`:	Customer’s payment habits and tendencies.
- `monthly_balance`:	The remaining balance in the customer’s account at the end of each month.
- `credit_score`:	The customer’s credit score (target variable: "Good," "Poor," "Standard").

**Initial thoughts**:
- Privacy concerns: `ssn`, `name` are sensitive data, we should not include them in the final model.
- Usefulness: `id`, `customer_id` are not useful for prediction, we should not include them in the final model.
- Time series: `month` is a time series data, but it does not contain year information, we should be careful when using it.
- Imbalance: `credit_score` is an imbalanced data, we should be careful when using it.
- Loan Type: `type_of_loan` may contain multiple types of loans, we should be careful when using it.

In [23]:
# Data loading
data_path = 'data/'
train_df = pd.read_csv(data_path + 'train.csv')
test_df = pd.read_csv(data_path + 'test.csv')

# Column Processing (lower case)
train_df.columns = [col.lower() for col in train_df.columns]
test_df.columns = [col.lower() for col in test_df.columns]

# Drop ID column
train_df = train_df.drop(['id','name', 'ssn'], axis=1)
test_df = test_df.drop(['id','name', 'ssn'], axis=1)

# Data overview
print(f"Train data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")
print(f"Prediction target: {train_df['credit_score'].value_counts()}")

C:\Users\29100\AppData\Local\Temp\ipykernel_156616\1528034292.py:3: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(data_path + 'train.csv')


Train data shape: (100000, 25)
Test data shape: (50000, 24)
Prediction target: credit_score
Standard    53174
Poor        28998
Good        17828
Name: count, dtype: int64


In [24]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 25 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   customer_id               100000 non-null  object 
 1   month                     100000 non-null  object 
 2   age                       100000 non-null  object 
 3   occupation                100000 non-null  object 
 4   annual_income             100000 non-null  object 
 5   monthly_inhand_salary     84998 non-null   float64
 6   num_bank_accounts         100000 non-null  int64  
 7   num_credit_card           100000 non-null  int64  
 8   interest_rate             100000 non-null  int64  
 9   num_of_loan               100000 non-null  object 
 10  type_of_loan              88592 non-null   object 
 11  delay_from_due_date       100000 non-null  int64  
 12  num_of_delayed_payment    92998 non-null   object 
 13  changed_credit_limit      100000 non-null  ob

In [25]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_id               50000 non-null  object 
 1   month                     50000 non-null  object 
 2   age                       50000 non-null  object 
 3   occupation                50000 non-null  object 
 4   annual_income             50000 non-null  object 
 5   monthly_inhand_salary     42502 non-null  float64
 6   num_bank_accounts         50000 non-null  int64  
 7   num_credit_card           50000 non-null  int64  
 8   interest_rate             50000 non-null  int64  
 9   num_of_loan               50000 non-null  object 
 10  type_of_loan              44296 non-null  object 
 11  delay_from_due_date       50000 non-null  int64  
 12  num_of_delayed_payment    46502 non-null  object 
 13  changed_credit_limit      50000 non-null  object 
 14  num_cr

# 1. Exploratory Data Analysis
> EDA is the process of visualizing and summarizing data in order to gain insights and make informed decisions. During the EDA, we will clean the data, visualize the data, and summarize the data.

## 1.1. Data Cleaning
For this dataset, we will not perform the the normal data cleaning process since weird values are present. It is necessary to mark the weird values and convert them to missing value, and handle them in the final stage.
## 1.1.1. Consistency Check

In [26]:
# Unique values in selected columns
total_df = pd.concat([train_df, test_df])
train_df_index = len(train_df)
selected_columns = [col for col in train_df.columns if col not in ['customer_id', 'credit_score']]
for col in selected_columns:
    print(f"Unique values for `{col}` : {total_df[col].unique()}")
    print(total_df[col].value_counts())
    print()

Unique values for `month` : ['January' 'February' 'March' 'April' 'May' 'June' 'July' 'August'
 'September' 'October' 'November' 'December']
month
January      12500
February     12500
March        12500
April        12500
May          12500
June         12500
July         12500
August       12500
September    12500
October      12500
November     12500
December     12500
Name: count, dtype: int64

Unique values for `age` : ['23' '-500' '28_' ... '326' '7910' '5826']
age
39       4198
32       4189
28       4173
26       4140
35       4130
         ... 
8655        1
4383_       1
2824        1
6365        1
6150        1
Name: count, Length: 2524, dtype: int64

Unique values for `occupation` : ['Scientist' '_______' 'Teacher' 'Engineer' 'Entrepreneur' 'Developer'
 'Lawyer' 'Media_Manager' 'Doctor' 'Journalist' 'Manager' 'Accountant'
 'Musician' 'Mechanic' 'Writer' 'Architect']
occupation
_______          10500
Lawyer            9899
Engineer          9562
Architect         9550
Mechan

In [27]:
# Check first 5 values in each column that are not numeric data
for col in selected_columns:
    print(f"{col} : {[value for value in list(total_df[col].unique()) if not str(value).replace(".", "").isdigit()][:5]}")

month : ['January', 'February', 'March', 'April', 'May']
age : ['-500', '28_', '34_', '30_', '24_']
occupation : ['Scientist', '_______', 'Teacher', 'Engineer', 'Entrepreneur']
annual_income : ['34847.84_', '30689.89_', '35547.71_', '34081.38_', '114838.41_']
monthly_inhand_salary : [np.float64(nan)]
num_bank_accounts : [np.int64(-1)]
num_credit_card : []
interest_rate : []
num_of_loan : ['-100', '0_', '3_', '2_', '5_']
type_of_loan : ['Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan', 'Credit-Builder Loan', 'Auto Loan, Auto Loan, and Not Specified', 'Not Specified', nan]
delay_from_due_date : [np.int64(-1), np.int64(-2), np.int64(-3), np.int64(-5), np.int64(-4)]
num_of_delayed_payment : [nan, '8_', '-1', '3_', '2_']
changed_credit_limit : ['_', '-2.01', '-1.01', '-3.01', '-1.2400000000000002']
num_credit_inquiries : [np.float64(nan)]
credit_mix : ['_', 'Good', 'Standard', 'Bad']
outstanding_debt : ['1328.93_', '1283.37_', '2797.17_', '3818.57_', '343.84_']
credit_u

From above observation, we could identify the following:
- Time Information: `month` has 12 values, it is categorical data, `credit_history_age` has different format - '22 Years and 1 Months', should be cleaned into months
- Irregular Format: `age`, `annual_income`, `num_of_loan`, `num_of_delayed_payment`, `outstanding_debt` and more columns might have irregular format (eg. 3.12_ / 5_), `credit_history_age` have year and month information, but the data might need to be cleaned into months (e.g. 22 Years and 1 Months)
- Irregular Values: `age`, `delay_from_due_date`, `num_of_loan` and more columns might have irregular values (eg. Negative values or large values)
- Missing/Unknown Values: Columns might have missing/unknown values, but in different formats (e.g. `_______`, `nan`)
- More Values in One Column: `type_of_loan` has more than 1 value in one column, need to be cleaned via one-hot encoding

In [28]:
print(sorted(total_df['annual_income'].unique()))

['10001727.0', '10003.495', '10003.495_', '10004.805', '100058.19', '100058.19_', '10007.815', '10007894.0', '10009.905', '100097.19', '100097.19_', '100100.22', '100100.22_', '100127.82', '100148.8', '100188.84', '100191.36000000002', '100191.36000000002_', '100192.32', '100192.32_', '10021.72', '100233.4', '10024.25', '10024.25_', '10024.955', '100241.67', '10025.2', '10025.2_', '100268.73', '10027.07', '10027.07_', '100284.24', '100284.24_', '10030.52', '100310.08', '100310.08_', '10034.55', '10034.55_', '10034.725', '10034.725_', '1003489.0', '100358.73', '100358.73_', '10037.5', '10037.735', '10037.735_', '10037.855', '10041.885', '10041990.0', '10042124.0_', '100433.58', '10046.24', '10046.24_', '10046425.0', '100465.13999999998', '100505.49', '10053569.0', '100536.2', '100536.2_', '100555.86000000002', '100555.86000000002_', '10060.015', '10062.445', '10062.445_', '10062.82', '10062.82_', '100666.59', '100674.66', '100674.66_', '10068.97', '10068.97_', '100681.41', '100686.12', 

In [29]:
# 1. Time
## Ordinal Encoding
ordinal_encoder = OrdinalEncoder(dtype='int64')
ordinal_encoder.fit(total_df[['month']])
total_df['month'] = ordinal_encoder.transform(total_df[['month']])
## credit_history_age year and month -> year * 12 + month
total_df['credit_history_age_in_month'] = total_df['credit_history_age'].apply(lambda x: int(float(str(x).split(' ')[0])) * 12 + int(float(str(x).split(' ')[3])) if pd.notnull(x) else np.nan)
total_df = total_df.drop(['credit_history_age'], axis=1)

# 2. Irregular Format: Correct format for unknown values - '_'or 'nan'
total_df['credit_mix'] = total_df['credit_mix'].apply(lambda x: np.nan if x == '_' else x)
total_df['changed_credit_limit'] = total_df['changed_credit_limit'].apply(lambda x: np.nan if x == '_' else x)
total_df['occupation'] = total_df['occupation'].apply(lambda x: np.nan if x.replace("_", "") == '' else x)
total_df['payment_behaviour'] = total_df['payment_behaviour'].apply(lambda x: np.nan if x == '!@9#%8' else x)
total_df['amount_invested_monthly'] = total_df['amount_invested_monthly'].apply(lambda x: np.nan if x == '__10000__' else x)

# 3. Irregular Format: Convert original data to numeric, while handling unknown values '_'
int_format_cols = ['age', 'num_of_loan', 'num_of_delayed_payment']
float_format_cols = ['annual_income', 'amount_invested_monthly', 'outstanding_debt', 'monthly_balance', 'changed_credit_limit']
for col in int_format_cols:
    total_df[col] = total_df[col].apply(lambda x: int(float(str(x).replace('_', ''))) if  x != '_' and pd.notnull(x) else np.nan)
for col in float_format_cols:
    total_df[col] = total_df[col].apply(lambda x: float(str(x).replace('_', '')) if  x != '_' and pd.notnull(x) else np.nan)

# 4. One-hot encoding
total_df['type_of_loan'] = total_df['type_of_loan'].str.replace(' and ', ' ', regex=False)
total_df['type_of_loan'] = total_df['type_of_loan'].str.split(', ')
total_df['type_of_loan'] = total_df['type_of_loan'].apply(lambda x: x if isinstance(x, list) else [])

mlb = MultiLabelBinarizer()
loan_type_encoded = pd.DataFrame(mlb.fit_transform(total_df['type_of_loan']),
                                 columns=[f"{cls.replace('-', '_').replace(' ', '_').lower()}" for cls in mlb.classes_],
                                 index=total_df.index)

total_df = pd.concat([total_df, loan_type_encoded], axis=1)
total_df = total_df.drop('type_of_loan', axis=1)

In [30]:
# # Check first 5 values in each column that are not numeric data
for col in total_df.columns:
    print(f"{col} : {[value for value in list(total_df[col].unique()) if not str(value).replace(".", "").isdigit()][:5]}")
total_df.info()

customer_id : ['CUS_0xd40', 'CUS_0x21b1', 'CUS_0x2dbc', 'CUS_0xb891', 'CUS_0x1cdb']
month : []
age : [np.int64(-500)]
occupation : ['Scientist', nan, 'Teacher', 'Engineer', 'Entrepreneur']
annual_income : []
monthly_inhand_salary : [np.float64(nan)]
num_bank_accounts : [np.int64(-1)]
num_credit_card : []
interest_rate : []
num_of_loan : [np.int64(-100)]
delay_from_due_date : [np.int64(-1), np.int64(-2), np.int64(-3), np.int64(-5), np.int64(-4)]
num_of_delayed_payment : [np.float64(nan), np.float64(-1.0), np.float64(-3.0), np.float64(-2.0)]
changed_credit_limit : [np.float64(nan), np.float64(-2.01), np.float64(-1.01), np.float64(-3.01), np.float64(-1.2400000000000002)]
num_credit_inquiries : [np.float64(nan)]
credit_mix : [nan, 'Good', 'Standard', 'Bad']
outstanding_debt : []
credit_utilization_ratio : []
payment_of_min_amount : ['No', 'NM', 'Yes']
total_emi_per_month : []
amount_invested_monthly : [np.float64(nan)]
payment_behaviour : ['High_spent_Small_value_payments', 'Low_spent_Larg

In [31]:
for col in total_df.columns:
    result = [value for value in list(total_df[col].unique()) if not str(value).replace(".", "").isdigit()]
    if len(result) > 0:
        print(f"{col} : {result}")

customer_id : ['CUS_0xd40', 'CUS_0x21b1', 'CUS_0x2dbc', 'CUS_0xb891', 'CUS_0x1cdb', 'CUS_0x95ee', 'CUS_0x284a', 'CUS_0x5407', 'CUS_0x4157', 'CUS_0xba08', 'CUS_0xa66b', 'CUS_0xc0ab', 'CUS_0x3e45', 'CUS_0x6c66', 'CUS_0xff4', 'CUS_0x33d2', 'CUS_0x6070', 'CUS_0xfdb', 'CUS_0x3553', 'CUS_0x4100', 'CUS_0x132f', 'CUS_0xa16e', 'CUS_0xac86', 'CUS_0x5b48', 'CUS_0xa5f9', 'CUS_0x4d43', 'CUS_0xb95f', 'CUS_0x3edc', 'CUS_0xbffe', 'CUS_0xb681', 'CUS_0x9a71', 'CUS_0x6048', 'CUS_0x95b5', 'CUS_0x4004', 'CUS_0xb101', 'CUS_0x9b94', 'CUS_0x4080', 'CUS_0x706a', 'CUS_0x42ac', 'CUS_0x571f', 'CUS_0xb5ff', 'CUS_0x6a1b', 'CUS_0x9bc1', 'CUS_0xc65', 'CUS_0xaedb', 'CUS_0x8e9b', 'CUS_0x609d', 'CUS_0x75c6', 'CUS_0x9b3c', 'CUS_0x22a3', 'CUS_0xb14', 'CUS_0xb986', 'CUS_0xbe1b', 'CUS_0x9ce6', 'CUS_0x6ffb', 'CUS_0x4315', 'CUS_0x3f5b', 'CUS_0x9d78', 'CUS_0x47db', 'CUS_0x8f17', 'CUS_0x7d0b', 'CUS_0x7504', 'CUS_0x774', 'CUS_0x9f70', 'CUS_0xab76', 'CUS_0x5cdf', 'CUS_0x2184', 'CUS_0xa156', 'CUS_0x74f2', 'CUS_0x308b', 'CUS_0x2827

## 1.1.2. Outlier Check

In [32]:
numeric_cols = total_df.select_dtypes(include=np.number).columns.tolist()
for col in total_df.columns:
    if col in numeric_cols:
        print(f"{col}: {sorted(list(total_df[col].unique()), reverse=False)[:5]}")
    else:   
        print(f"{col}: {list(total_df[col].unique())[:5]}")

customer_id: ['CUS_0xd40', 'CUS_0x21b1', 'CUS_0x2dbc', 'CUS_0xb891', 'CUS_0x1cdb']
month: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4)]
age: [np.int64(-500), np.int64(14), np.int64(15), np.int64(16), np.int64(17)]
occupation: ['Scientist', nan, 'Teacher', 'Engineer', 'Entrepreneur']
annual_income: [np.float64(7005.93), np.float64(7006.035), np.float64(7006.52), np.float64(7011.685), np.float64(7012.31)]
monthly_inhand_salary: [np.float64(303.6454166666666), np.float64(319.55625), np.float64(332.12833333333333), np.float64(333.5966666666667), np.float64(355.20833333333337)]
num_bank_accounts: [np.int64(-1), np.int64(0), np.int64(1), np.int64(2), np.int64(3)]
num_credit_card: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4)]
interest_rate: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5)]
num_of_loan: [np.int64(-100), np.int64(0), np.int64(1), np.int64(2), np.int64(3)]
delay_from_due_date: [np.int64(-5), np.int64(-4), np.int64(-3), np.i

In [33]:
selected_columns = ['month', 'age', 'annual_income', 'monthly_inhand_salary',
       'num_bank_accounts', 'num_credit_card', 'interest_rate', 'num_of_loan',
       'delay_from_due_date', 'num_of_delayed_payment', 'changed_credit_limit',
       'num_credit_inquiries', 'outstanding_debt', 'credit_utilization_ratio',
       'total_emi_per_month', 'amount_invested_monthly', 'monthly_balance',
       'credit_history_age_in_month']

print(total_df[selected_columns].describe())

def detect_outlier(df, selected_columns):
    for col in selected_columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        outliers = df[(df[col] < Q1 - 1.5*IQR) | (df[col] > Q3 + 1.5*IQR)][col]
        print(f"\n{col}:")
        print(f"Outlier Number: {len(outliers)}")
        if len(outliers) > 0:
            print("Outlier Example:")
            print(outliers.head())

detect_outlier(total_df, selected_columns)

               month            age  annual_income  monthly_inhand_salary  \
count  150000.000000  150000.000000   1.500000e+05          127500.000000   
mean        5.500000     110.337940   1.730552e+05            4190.115139   
std         3.452064     684.066779   1.404215e+06            3180.489657   
min         0.000000    -500.000000   7.005930e+03             303.645417   
25%         2.750000      25.000000   1.945549e+04            1625.265833   
50%         5.500000      33.000000   3.757861e+04            3091.000000   
75%         8.250000      42.000000   7.279690e+04            5948.454596   
max        11.000000    8698.000000   2.419806e+07           15204.633333   

       num_bank_accounts  num_credit_card  interest_rate    num_of_loan  \
count      150000.000000    150000.000000  150000.000000  150000.000000   
mean           17.006940        22.623447      71.234907       3.141093   
std           117.069476       129.143006     461.537193      63.910655   
min   

We could observe some weird value in the data.
- Negative values: `age`, `num_bank_accounts`, `num_of_loan`, `num_of_delayed_payment`, `monthly_balance` should not have negative value
- Large values: 
    - `age`: normally, age should not exceed 120
    - `annual_income`: normally, annual_income should not exceed 1e7
    - `num_bank_accounts`: normally, bank account should not exceed 100
    - `interest_rate`: normally, interest_rate should not exceed 100
    - `num_of_loan`: normally, number of loan should not exceed 100
    - `num_of_delayed_payment`: normally, number of delayed payment should not exceed 100
    - ....More could been observe from describe()
- Mismatch in Age: `age` should be at least larger than `credit_history_age_in_month / 12`

Rest columns seems to be normal compared to other extremes.

In [34]:
# Negative Values
total_df['age'] = total_df['age'].apply(lambda x: x if x >= 0 and x <= 120 else np.nan)
total_df['num_bank_accounts'] = total_df['num_bank_accounts'].apply(lambda x: x if x >= 0 and x <= 100 else np.nan)
total_df['num_of_loan'] = total_df['num_of_loan'].apply(lambda x: x if x >= 0 and x <= 100 else np.nan)
total_df['num_of_delayed_payment'] = total_df['num_of_delayed_payment'].apply(lambda x: x if x >= 0 and x <= 100 else np.nan)
total_df['monthly_balance'] = total_df['monthly_balance'].apply(lambda x: x if x >= -100000 and x <= 10000000 else np.nan)
# Large Values - no negative values
total_df['annual_income'] = total_df['annual_income'].apply(lambda x: x if x <= 10000000 else np.nan)
total_df['interest_rate'] = total_df['interest_rate'].apply(lambda x: x if x <= 100 else np.nan)
total_df['num_credit_card'] = total_df['num_credit_card'].apply(lambda x: x if x <= 100 else np.nan)
total_df['num_credit_inquiries'] = total_df['num_credit_inquiries'].apply(lambda x: x if x <= 100 else np.nan)
# Weird Values
total_df['credit_history_age_in_month'] = total_df.apply(lambda row: row['credit_history_age_in_month'] 
                                                        if row['age'] >= row['credit_history_age_in_month'] / 12 
                                                        else np.nan, axis=1)

In [35]:
selected_columns = ['month', 'age', 'annual_income', 'monthly_inhand_salary',
       'num_bank_accounts', 'num_credit_card', 'interest_rate', 'num_of_loan',
       'delay_from_due_date', 'num_of_delayed_payment', 'changed_credit_limit',
       'num_credit_inquiries', 'outstanding_debt', 'credit_utilization_ratio',
       'total_emi_per_month', 'amount_invested_monthly', 'monthly_balance',
       'credit_history_age_in_month']

print(total_df[selected_columns].describe())

def detect_outlier(df, selected_columns):
    for col in selected_columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        outliers = df[(df[col] < Q1 - 1.5*IQR) | (df[col] > Q3 + 1.5*IQR)][col]
        print(f"\n{col}:")
        print(f"Outlier Number: {len(outliers)}")
        if len(outliers) > 0:
            print("Outlier Example:")
            print(outliers.head())

detect_outlier(total_df, selected_columns)

               month            age  annual_income  monthly_inhand_salary  \
count  150000.000000  145830.000000   1.491010e+05          127500.000000   
mean        5.500000      33.494686   7.069578e+04            4190.115139   
std         3.452064      10.794240   3.675344e+05            3180.489657   
min         0.000000      14.000000   7.005930e+03             303.645417   
25%         2.750000      25.000000   1.939650e+04            1625.265833   
50%         5.500000      33.000000   3.723350e+04            3091.000000   
75%         8.250000      42.000000   7.211625e+04            5948.454596   
max        11.000000     120.000000   9.976017e+06           15204.633333   

       num_bank_accounts  num_credit_card  interest_rate    num_of_loan  \
count      148116.000000    146765.000000  147025.000000  143467.000000   
mean            5.405392         5.611011      14.542942       3.551444   
std             2.996474         3.047392       8.785156       2.662183   
min   

## 1.1.3. Duplicate Check

In [36]:
# Duplicate
print(train_df[['customer_id', 'month']].duplicated().sum())
print(test_df[['customer_id', 'month']].duplicated().sum())

0
0


## 1.1.4. Missing Values
We now have two types of missing values:

1.Original Missing Values: These are the missing values that are present in the original dataset. They can be of any type, such as numerical, categorical, or text.

2.Imputed Missing Values: These are the missing values that are created by the outlier limitation process. They are created by replacing the outlier values with the upper or lower limit of the outlier detection range.

We should try to match the missing values in the original dataset with the other value in the dataset (under same customer_id). 

In [37]:
# Missing Values - Checked All Columns (Matched)
missing_df = pd.concat([
    pd.DataFrame(train_df.isnull().sum(), columns=['train_miss']),
    pd.DataFrame(test_df.isnull().sum(), columns=['test_miss']),
    pd.DataFrame(total_df.isnull().sum(), columns=['total_miss']),
], axis=1).reset_index()
missing_df[(missing_df['train_miss'] > 0) | (missing_df['test_miss'] > 0) | (missing_df['total_miss'] > 0)]

,index,train_miss,test_miss,total_miss
2,age,0.0,0.0,4170.0
3,occupation,0.0,0.0,10500.0
4,annual_income,0.0,0.0,899.0
5,monthly_inhand_salary,15002.0,7498.0,22500.0
6,num_bank_accounts,0.0,0.0,1884.0
7,num_credit_card,0.0,0.0,3235.0
8,interest_rate,0.0,0.0,2975.0
9,num_of_loan,0.0,0.0,6533.0
10,type_of_loan,11408.0,5704.0,NaN
12,num_of_delayed_payment,7002.0,3498.0,12545.0


In [38]:
# Some of missing value could be due to that customer forget to write some information
## We could check if all customers have 12 months of data, if not, it could represent forget to write / other reasons
missing_df = total_df.groupby(['customer_id']).count().reset_index()
unsatisfied_cols = []
unsatisfied_ids = set()
for col in missing_df.columns:
    if col != 'customer_id':
        unsatisfied_rows = missing_df[missing_df[col] < 12]
        if len(unsatisfied_rows) > 0:
            unsatisfied_customer_ids = set(unsatisfied_rows['customer_id'].tolist())
            unsatisfied_cols.append(col)
            unsatisfied_ids = unsatisfied_ids.union(unsatisfied_customer_ids)
print(len(unsatisfied_ids))
print(unsatisfied_cols)

12500
['age', 'occupation', 'annual_income', 'monthly_inhand_salary', 'num_bank_accounts', 'num_credit_card', 'interest_rate', 'num_of_loan', 'num_of_delayed_payment', 'changed_credit_limit', 'num_credit_inquiries', 'credit_mix', 'amount_invested_monthly', 'payment_behaviour', 'monthly_balance', 'credit_score', 'credit_history_age_in_month']


In [39]:
# Check if there are columns that have only one value (and null), we can then copy the one value to null values
def find_customers_with_many_values(df, target_col, threshold=2):
    unique_values = df.groupby('customer_id')[target_col].apply(lambda x: x.unique())
    result_df = unique_values.reset_index()
    result_df.columns = ['customer_id', 'unique_values']
    result_df['num_unique'] = result_df['unique_values'].apply(len)
    filtered = result_df[result_df['num_unique'] >= threshold]
    return filtered
# Fill null values with the value that is not null for each customer
def fill_single_value_per_customer(df, target_col):
    # Find value that are not null for each customer
    single_value = (
        df[~df[target_col].isna()]
        .groupby('customer_id')[target_col]
        .agg(lambda x: x.unique()[0])  # Get the first unique value
        .rename(f'{target_col}_filled')
    )
    # Merge with original dataframe and fill null values
    df = df.merge(single_value, on='customer_id', how='left')
    df[target_col] = df[target_col].fillna(df[f'{target_col}_filled'])
    df.drop(columns=[f'{target_col}_filled'], inplace=True)
    return df

# Check and Paste columns that have only one value (and null)
one_value_cols = []
for col in unsatisfied_cols:
    filtered = find_customers_with_many_values(total_df, col)
    print(f"{col}: {filtered['num_unique'].unique()}")
    if len(filtered['num_unique'].unique()) == 1 and filtered['num_unique'].unique()[0] == 2:
        one_value_cols.append(col)

print(f"Columns with only one value (and Null Value): {one_value_cols}")
# for col in one_value_cols:
#     total_df = fill_single_value_per_customer(total_df, col)

age: [2 3 4]
occupation: [2]
annual_income: [2 3 4]
monthly_inhand_salary: [2 3]
num_bank_accounts: [2 3 4]
num_credit_card: [2 3 4]
interest_rate: [2 3]
num_of_loan: [2 3]
num_of_delayed_payment: [4 6 3 5 7 2 8]
changed_credit_limit: [3 5 4 2 6 7 8 9]
num_credit_inquiries: [2 3 4]
credit_mix: [2]
amount_invested_monthly: [12  9 11 10  8  7]
payment_behaviour: [6 5 7 4 3 2]
monthly_balance: [12 11  9 10  8  6  7]
credit_score: [3 2 4]
credit_history_age_in_month: [12 11 10  9  2  6  8  3  5  7  4]
Columns with only one value (and Null Value): ['occupation', 'credit_mix']


In [53]:
def fill_grouped_missing(df, group_col, numeric_cols, categorical_cols):
    # Numeric Columns Could Fill with Group Minimum
    for col in numeric_cols:
        if df[col].isnull().any():
            df[col] = df.groupby(group_col)[col].transform(lambda x: x.fillna(x.min()))
    # Categorical Columns Could Fill with Group Mode
    for col in categorical_cols:
        if df[col].isnull().any():
            df[col] = df.groupby(group_col)[col].transform(
                lambda x: x.fillna(x.mode()[0] if not x.mode().empty else np.nan))
    return df

# Numeric Columns Could Fill with Group Minimum
numeric_cols = ['age', 'annual_income', 'monthly_inhand_salary', 'num_bank_accounts', 'num_credit_card', 'interest_rate', 'num_of_loan', 'num_of_delayed_payment', 'changed_credit_limit', 'num_credit_inquiries', 'amount_invested_monthly', 'monthly_balance', 'credit_history_age_in_month']
# Categorical Columns Could Fill with Group Mode
categorical_cols = ['occupation', 'credit_mix', 'payment_behaviour']
new_total_df = fill_grouped_missing(total_df, group_col='customer_id', numeric_cols = numeric_cols, categorical_cols = categorical_cols)

In [54]:
# Missing Values - Checked All Columns (Matched)
missing_df = pd.concat([
    pd.DataFrame(train_df.isnull().sum(), columns=['train_miss']),
    pd.DataFrame(test_df.isnull().sum(), columns=['test_miss']),
    pd.DataFrame(new_total_df.isnull().sum(), columns=['total_miss']),
], axis=1).reset_index()
missing_df[(missing_df['train_miss'] > 0) | (missing_df['test_miss'] > 0) | (missing_df['total_miss'] > 0)]

,index,train_miss,test_miss,total_miss
5,monthly_inhand_salary,15002.0,7498.0,0.0
10,type_of_loan,11408.0,5704.0,NaN
12,num_of_delayed_payment,7002.0,3498.0,0.0
14,num_credit_inquiries,1965.0,1035.0,0.0
18,credit_history_age,9030.0,4470.0,NaN
21,amount_invested_monthly,4479.0,2271.0,0.0
23,monthly_balance,1200.0,562.0,0.0
24,credit_score,0.0,NaN,50000.0
25,credit_history_age_in_month,NaN,NaN,16692.0


After the match, `credit_history_age_in_month` still have missing values.

We could either drop the rows with missing values or impute them.

In [57]:
new_train_df = new_total_df[:train_df_index].dropna()
features = new_total_df.columns.tolist()
features.remove('credit_score')
new_test_df = new_total_df[train_df_index:][features].dropna()

In [59]:
print(new_train_df.shape)
print(new_test_df.shape)

(88872, 33)
(44436, 32)


In [61]:
new_train_df.describe().T

,count,mean,std,min,25%,50%,75%,max
month,88872.0,4.250000,2.633928,0.000000,2.500000,4.500000,6.250000,8.000000e+00
age,88872.0,34.773584,10.464625,14.000000,27.000000,35.000000,43.000000,1.180000e+02
annual_income,88872.0,68009.288553,354724.223518,7005.930000,19013.285000,36637.100000,70391.920000,9.950314e+06
monthly_inhand_salary,88872.0,4101.173085,3135.331592,303.645417,1589.923125,3039.035833,5829.833333,1.520463e+04
num_bank_accounts,88872.0,5.533599,2.953381,0.000000,4.000000,6.000000,8.000000,1.000000e+02
num_credit_card,88872.0,5.705171,3.020968,0.000000,4.000000,6.000000,7.000000,1.000000e+02
interest_rate,88872.0,15.086540,8.904851,1.000000,8.000000,14.000000,21.000000,1.000000e+02
num_of_loan,88872.0,3.700468,2.714072,0.000000,2.000000,3.000000,6.000000,1.000000e+02
delay_from_due_date,88872.0,21.852394,15.176654,-5.000000,10.000000,19.000000,29.000000,6.700000e+01
num_of_delayed_payment,88872.0,13.487735,6.309690,0.000000,9.000000,14.000000,18.000000,9.800000e+01


In [3]:
# Final Convertion
object_columns = new_total_df.select_dtypes(include=['object']).columns.tolist()
for col in object_columns:
    print(f"{col}: {new_total_df[col].unique()}")

NameError: name 'new_total_df' is not defined

In [ ]:
new_total_df['occupation'].value_counts(normalize=True)

NameError: name 'new_total_df' is not defined

In [ ]:
# # Convert from categorical to numerical
# # 1. customer_id
# new_total_df = new_total_df.drop('customer_id', axis=1)

# # 2. occupation
# occupation_map = new_total_df.groupby('occupation')['credit_score'].apply(lambda x: (x == 'Good').mean() - (x == 'Poor').mean()).to_dict()
# new_total_df['occupation_encoded'] = new_total_df['occupation'].map(occupation_map)

# # 3. credit_mix
# credit_mix_order = {'Bad':0, 'Standard':1, 'Good':2}
# new_total_df['credit_mix'] = pd.Categorical(new_total_df['credit_mix'], categories=credit_mix_order.keys()).codes

# # 4. minimum_payment
# pmt_map = {'No':0, 'NM':1, 'Yes':2}
# new_total_df['payment_of_min_amount'] = new_total_df['payment_of_min_amount'].replace(pmt_map)

# # 5. payment_behaviour
# new_total_df[['spending_level', 'payment_size']] = new_total_df['payment_behaviour'].str.split('_', n=1, expand=True)
# new_total_df['spending_level'] = new_total_df['spending_level'].map({'High':2, 'Low':0, 'Medium':1})
# new_total_df['payment_size'] = new_total_df['payment_size'].map({'Large_value_payments':2, 
#                                             'Medium_value_payments':1,
#                                             'Small_value_payments':0})

# # 6. credit_score mapping
# score_map = {'Poor':0, 'Standard':1, 'Good':2}
# new_total_df['credit_score'] = new_total_df['credit_score'].map(score_map)

In [ ]:
# ======== PyTorch自编码器填补模块 ========
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

class FinancialAutoencoder(nn.Module):
    def __init__(self, input_dim):
        super(FinancialAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.Tanh()
        )
        self.decoder = nn.Sequential(
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, input_dim)
        )
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

def torch_ae_imputation(data, features_to_impute, epochs=100, batch_size=256):
    # 特征选择与预处理
    # related_features = ['age', 'annual_income', 'num_credit_inquiries',
    #                    'credit_score', 'num_of_loan', 'monthly_inhand_salary',
    #                    'interest_rate', 'num_bank_accounts', 'credit_history_age_in_month']
    related_features = data.columns.tolist()
    # related_features.remove('customer_id')
    related_features.remove('credit_score')
    
    # 初步填充
    temp_df = data[related_features].copy()
    imp = SimpleImputer(strategy='median')
    temp_df = pd.DataFrame(imp.fit_transform(temp_df), columns=related_features)
    
    # 数据标准化
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(temp_df)
    
    # 转换为PyTorch张量
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    tensor_data = torch.FloatTensor(scaled_data).to(device)
    
    # 初始化模型
    model = FinancialAutoencoder(input_dim=len(related_features)).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # 训练循环
    best_loss = float('inf')
    patience = 10
    no_improve = 0
    
    # 自动批处理
    dataset = torch.utils.data.TensorDataset(tensor_data, tensor_data)
    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for batch_x, _ in loader:
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_x)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # 早停机制
        with torch.no_grad():
            model.eval()
            val = model(tensor_data)
            val_loss = criterion(val, tensor_data).item()
            
            if val_loss < best_loss:
                best_loss = val_loss
                no_improve = 0
                best_weights = model.state_dict()
            else:
                no_improve += 1
                
            if no_improve >= patience:
                print(f'Early stopping at epoch {epoch}')
                break
    
    # 加载最佳权重
    model.load_state_dict(best_weights)
    
    # 生成预测
    with torch.no_grad():
        reconstructed = model(tensor_data).cpu().numpy()
    reconstructed = scaler.inverse_transform(reconstructed)
    
    # 替换缺失值
    for i, feature in enumerate(related_features):
        if feature in features_to_impute:
            mask = data[feature].isnull()
            data.loc[mask, feature] = reconstructed[mask, i]
    
    return data

# ======== 执行填补 ========
# 检查GPU可用性
print("可用GPU设备:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "使用CPU")

# 应用PyTorch版本填补
impute_features = ['credit_history_age_in_month', 'monthly_inhand_salary']
total_df = torch_ae_imputation(total_df, impute_features, epochs=150)

可用GPU设备: NVIDIA GeForce RTX 3070 Laptop GPU


ValueError: Cannot use median strategy with non-numeric data:
could not convert string to float: 'Scientist'

In [ ]:
print(new_train_df.shape)
print(new_test_df.shape)

(88872, 33)
(44436, 32)


In [ ]:
print(new_train_df.shape)
print(new_test_df.shape)

(88872, 33)
(44436, 32)


In [ ]:
print(new_train_df.shape)
print(new_test_df.shape)

(88872, 33)
(44436, 32)


In [ ]:
print(new_train_df.shape)
print(new_test_df.shape)

(88872, 33)
(44436, 32)


In [ ]:
print(new_train_df.shape)
print(new_test_df.shape)

(88872, 33)
(44436, 32)


In [ ]:
# selected_columns = total_df.select_dtypes(include=['float64', 'int64']).columns.tolist()
# selected_columns = [column for column in selected_columns if 'loan' not in column and 'not_specified' not in column]
# selected_columns = selected_columns + ['num_of_loan']
# print(selected_columns)

In [ ]:
# # Generate subplots for selected columns, matplotlib
# fig, axes = plt.subplots(nrows=len(selected_columns), ncols=1, figsize=(8, 4 * len(selected_columns)))

# # Iterate over selected columns and plot histograms
# for i, column in enumerate(selected_columns):
#     ax = axes[i]
#     ax.hist(total_df[column])
#     ax.set_xlabel(column)
#     ax.set_ylabel('Frequency')
#     ax.set_title(f'Histogram of {column}')

# # Adjust spacing between subplots
# plt.tight_layout()

# # Display the plots
# plt.show()

In [ ]:
# To proceed with the data cleaning (including the potential imputation of missing values), 